# RAG(Retrieval-Augmented Generation)

## 0. 미션
참조
- 정보: https://cloud.google.com/vertex-ai/docs/generative-ai/open-models/use-gemma?hl=ko
- 2b instruction tuning: https://huggingface.co/google/gemma-1.1-2b-it
- SentenceTransformers: https://www.sbert.net/

미션
- 질문에 대해서 적절한 문서를 검색하고, 검색된 문서에 근거해서 답변하는 RAG 챗봇을 만들어봅니다.
- 검색된 문서에 근거해서 답변하는 기능은 구글의 gemma-2b-it SLLM을 사용합니다.

## 1. 라이브러리 설치 (최초 한번만 실행)
- 라이브러리는 colab이 최초 실행 또는 종료 후 실행된 경우 한번만 실행하면 됩니다.
- GPU 메모리 부족등의 이유로 colab 세션을 다시 시작한 경우는 설치할 필요 없습니다.
- colab 세션을 다시 시작하려면 '런타임' >> '세션 다시 시작'을 선택하세요.

In [ ]:
!pip install -qq -U transformers accelerate
!pip install -qq datasets
!pip install -qq peft
!pip install -qq bitsandbytes
!pip install -qq trl
!pip install -qq sentence-transformers

## 2. 구글 드라이브 연결 (최초 한번만 실행)
- 구글 드라이브는 데이터 저장 및 학습 결과를 저장하기 위해서 사용합니다.
- 구글 드라이브는 colab이 최초 실행 또는 종료 후 실행된 경우 한번 만 연결하면 됩니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## *3. 환경 (매번 필수 실행)
- 환경은 colab 세션을 처음 시작하거나 다시 시작한 경우 실행되어야 합니다.
- 프로젝트 진행에 필요한 환경을 설정합니다.

### 3.1. 라이브러리 Import

In [ ]:
import os
import glob
import json

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import torch
from sentence_transformers import SentenceTransformer
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          pipeline)

### 3.2. 환경정보 설정
- HF_TOKEN:
  - Hugging Face 인증을 위한 Token
  - 아래 URL에 접속해서 'User Access Token'을 생성하고 복사해서 Token에 입력하세요.
  - https://huggingface.co/settings/tokens
- WORKSPACE
  - 학습 데이터 및 학습결과를 저장하기 위한 경로입니다.
  - 필요할 경우 적당한 경로로 변경할 수 있습니다.
  - 경로를 변경 할 경우 전체 경로에 공백이 포함되지 않도록 주의해 주세요.
- EMBED_MODEL_ID
  - 검색을 위한 SentenceTransformer 입니다.
  - 서울대학교 컴퓨터언어학_자연어처리 연구실에서 공개한 모델입니다.
  - https://huggingface.co/snunlp/KR-SBERT-V40K-klueNLI-augSTS
- SLLM_MODEL_ID
  - 문서에 근거해서 답변 기능을 위한 SLLM 입니다.
  - 구글에서 공개한 gemma-2b를 Instruction tunned한 버전입니다.
  - https://huggingface.co/google/gemma-1.1-2b-it
- CHUNK_FN
  - 문서를 일정한 단위로 분할해서 저장된 파일 이름


In [ ]:
# access token을 복사하세요.
HF_TOKEN = ""

In [ ]:
WORKSPACE = '/content/drive/MyDrive/nlp-practice'
EMBED_MODEL_ID = 'snunlp/KR-SBERT-V40K-klueNLI-augSTS'
SLLM_MODEL_ID = 'google/gemma-1.1-2b-it'
CHUNK_FN = f"{WORKSPACE}/data/wiki_dump.json"

## 4. SentenceTransformers를 이용한 검색 tutorial
- SentenceTransformers를 이용해서 문장을 특징벡터로 변환하고 벡터의 비교를 이용해 유사한 문서를 검색합니다.

### 4.1. SentenceTransformers를 위한 전처리
- CHUNK_FN의 전체 문서를 사용하세요.

In [ ]:
# chunk list
chunk_list = []
with open(CHUNK_FN, encoding="utf-8") as f:
    chunk_list = json.load(f)
len(chunk_list)

In [ ]:
chunk_list[2]

### 4.2. SentenceTransformers 검색 함수
- SentenceTransformers를 활용한 검색 기능을 함수로 구현하고 실험합니다.

In [ ]:
# SentenceTransformer 모델 생성
embed_model = SentenceTransformer(EMBED_MODEL_ID)

In [ ]:
# 빠른 확인을 위해 1000개 chunk 만 사용
chunk_docs = []
for chunk in chunk_list[:1000]:
    title = chunk['metadata']['title']
    document = chunk['document']
    chunk_docs.append(f"{title}\n{document}")
len(chunk_docs)

In [ ]:
# chunk embeddings 생성
chunk_embeddings = embed_model.encode(chunk_docs, normalize_embeddings=True)
chunk_embeddings.shape

In [ ]:
# query embedding 생성
query = "지미 카터 대통령이 졸업한 대학교는?"
query_embedding = embed_model.encode(query, normalize_embeddings=True)
query_embedding.shape

In [ ]:
# 유사도 비교 (코싸인 유사도)
doc_scores = np.dot(chunk_embeddings, query_embedding)
doc_scores.shape

In [ ]:
# 유사도가 높은 순으로 정렬
rank = np.argsort(-doc_scores)
rank.shape

In [ ]:
# 상위 5개만 출력
top_n = 5
for i in rank[:top_n]:
    print("*" * 10, doc_scores[i], "*" * 10)
    print(chunk_list[i])

In [ ]:
# 검색 함수 정의
def query_sentence_transformer(embed_model, chunk_embeddings, query, top_n=5):
    query_embedding = embed_model.encode(query)
    # score 계산
    doc_scores = np.dot(chunk_embeddings, query_embedding)
    # score 순서로 정렬
    rank = np.argsort(-doc_scores)
    # top-n
    query_result = []
    for i in rank[:top_n]:
        query_result.append((i, doc_scores[i]))
    return query_result

In [ ]:
# 대화형 검색
while True:
    query = input('검색 > ')
    query = query.strip()
    if len(query) == 0:
        break
    query_result = query_sentence_transformer(embed_model, chunk_embeddings, query)
    for i, score in query_result:
        print(f'---- score: {score} ----')
        print(chunk_list[i])
        print()

## 5. SLLM RAG tutorial (재시작 필요)
- SLLM에 질문과 근거 문서를 함께 입력하고 질문에 맞는 답변을 근거 문서로 부터 하도록 하는 과정을 이해하기 위한 과정입니다.
- 이 과정을 시작하기 전 colab 세션을 다시 시작하세요.
- colab 세션을 다시 시작해야 하는 이유는 LLM의 model의 크기가 너무 크기 때문에 GPU의 메모리를 초기화 하기 위해서 입니다.

### 5.1. model load with 4 bits
- 2B token을 가진 gemma를 그냥 로딩할 경우는 약 9G의 GPU vRAM이 필요합니다.
- 4bit 양자화를 할 경우 2.2G의 GPU vRAM 필요.

In [ ]:
# declare 4 bits quantize
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
# load 4 bits model
sllm_model = AutoModelForCausalLM.from_pretrained(SLLM_MODEL_ID,
                                             device_map='auto',
                                             quantization_config=quantization_config,
                                             token=HF_TOKEN)
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(SLLM_MODEL_ID,
                                          token=HF_TOKEN)
tokenizer.padding_side = 'right'

### 5.2. pipeline
- https://huggingface.co/docs/transformers/main_classes/pipelines
- huggingface에서 inference를 쉽게 하기 위해 정의한 라이브러리.

In [ ]:
# pipeline
pipe = pipeline("text-generation",
                model=sllm_model,
                tokenizer=tokenizer,
                max_new_tokens=512)
pipe

### 5.3. sllm prompt

In [ ]:
# 검색어
query = "지미 카터 대통령이 졸업한 대학교는?"

# sentence transformer 검색결과
query_docs = [
    "지미 카터\n제임스 얼 “지미” 카터 주니어(, 1924년 10월 1일~)는 민주당 출신 미국의 제39대 대통령 (1977-81)이다.\n생애.\n어린 시절.\n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n조지아 공과대학교를 졸업하였고, 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 \"땅콩 농부\" (Peanut Farmer)로 알려졌다.\n정계 입문.\n1962년 조지아주 상원 의원 선거에서 낙선하였으나, 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사 선거에서 당선됐다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.\n대통령 재임.\n1976년 미합중국 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책을 내세워서 많은 지지를 받았으며 제럴드 포드 대통령을 누르고 당선되었다.\n카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.",
    "지미 카터\n1979년~1980년 대한민국의 정치적 격변기 당시의 대통령이었던 그는 이에 대해 애매한 태도를 보였고, 이는 후에 대한민국 내에서 고조되는 반미 운동의 한 원인이 됐다. 10월 26일, 박정희 대통령이 김재규 중앙정보부장에 의해 살해된 것에 대해 그는 이 사건으로 큰 충격을 받았으며, 사이러스 밴스 국무장관을 조문사절로 파견했다. 12·12 군사 반란과 5.17 쿠데타에 대해 초기에는 강하게 비난했으나, 미국 정부가 신군부를 설득하는데, 한계가 있었고 결국 묵인하는 듯한 태도를 보이게 됐다.\n퇴임 이후.\n퇴임 이후 민간 자원을 적극 활용한 비영리 기구인 카터 재단을 설립한 뒤 민주주의 실현을 위해 제 3세계의 선거 감시 활동 및 기니 벌레에 의한 드라쿤쿠르스 질병 방재를 위해 힘썼다. 미국의 빈곤층 지원 활동, 사랑의 집짓기 운동, 국제 분쟁 중재 등의 활동도 했다.\n카터는 카터 행정부 이후 미국이 북핵 위기, 코소보 전쟁, 이라크 전쟁과 같이 미국이 군사적 행동을 최후로 선택하는 전통적 사고를 버리고 군사적 행동을 선행하는 행위에 대해 깊은 유감을 표시 하며 미국의 군사적 활동에 강한 반대 입장을 보이고 있다.",
    "지미 카터\n카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.\n외교 정책.\n카터는 이집트와 이스라엘을 조정하여 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다. 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 그러나 1979년, 양국 간의 평화조약이 백악관에서 이루어졌다.\n소련과 제2차 전략 무기 제한 협상(SALT II)에 조인했다.\n카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으며, 취임 이후 계속해서 도덕정치를 내세웠다.\n임기 말, 소련의 아프가니스탄 침공 사건으로 인해 1980년 하계 올림픽에 반공국가들의 보이콧을 하였다.\n그는 주이란 미국 대사관 인질 사건의 인질 구출 실패로 인한 원인으로, 1980년 제40대 대통령 선거에서 공화당의 로널드 레이건에게 패배하며 재선에 실패하였다.\n대한민국과의 관계.\n지미 카터는 대한민국과의 관계에서도 중요한 영향을 미쳤던 대통령 중 하나다. 인권 문제와 주한미군 철수 문제로 한때 한미 관계가 불편하기도 했다. 1978년 대한민국에 대한 조선민주주의 인민공화국의 위협에 대비해 한미연합사를 창설하면서, 1982년까지 3단계에 걸쳐 주한미군을 철수하기로 했다. 그러나 주한미군사령부와 정보기관·의회의 반대에 부딪혀 주한미군은 완전철수 대신 6,000명을 감축하는 데 그쳤다. 또한 박정희 정권의 인권 문제 등과의 논란으로 불협화음을 냈으나, 1979년 6월 하순, 대한민국을 방문했는데 관계가 다소 회복되었다.",
    "노무현\n1953년에 진영대창국민학교에 입학하였고 학업 성적은 우수했으나 가난으로 결석이 잦았다고 한다. 6학년 때 담임교사의 권유로 전교 학생회장을 맡았다. 1959년 3월에 진영중학교에 진학했다. 중학교 재학 당시 노무현은 입학금이 없어 중학교는 외상으로 입학하였다고 한다. 1학년 말 제4대 정·부통령 선거를 앞두고 당시 이승만 대통령의 생일을 기념하는 교내 글짓기 대회가 열리자 노무현은 백지동맹을 일으키다가 정학을 당하였다. 집안 형편이 어려워져 중학교를 1년간 휴학한 뒤 부일장학회의 장학금을 얻어 가까스로 중학교에 다니다가 1963년에 가까스로 졸업하고 부산상고에 진학하여 1966년에 졸업하였다.\n청년기.\n고등학교 졸업 후 농업협동조합의 입사 시험에 응시했으나 낙방하고, 한 어망 제조업체에 취직하였으나 최저 생계비에도 미치지 못하는 임금과 다쳐도 치료비조차 주지 않는 고용주의 비정함에 실망하여 결국 그만두었다. 이후 막노동과 사시 공부를 병행하였다.\n1968년 군에 입대하여 제12사단 을지부대에서 육군 상병으로 만기전역하였다. 1972년 27세에 권양숙과 결혼하였고 1973년에 아들 건호를, 1975년에 딸 정연을 낳았다. 1975년 3월 30세에 제17회 사법시험에 합격하였다. 이는 4번째 도전에서 이루어진 성과였고 노무현은 제17회 사법시험에서 유일하게 고졸 출신 합격자였다. 이후 1977년 대전지방법원의 판사로 임용되었으나 5개월 만에 사직하였다.",
    "데니스 리치\n데니스 매캘리스터 리치(, 1941년 9월 9일~2011년 10월 12일)는 미국의 저명한 컴퓨터과학자이자 현대 컴퓨터과학의 선구자이다. C와 유닉스의 개발자로 알려져 있다.\n생애.\n미국의 뉴욕주 브롱크스빌(Bronxville)에서 태어났으며, 1968년 하버드 대학교에서 응용수학 박사학위를 얻었다. 1968년부터 벨 연구소 컴퓨터 연구 센터에서 일했다. 2007년 루슨트 테크놀로지의 시스템 소프트웨어 연구부장으로 은퇴했다. 홀로 살고 있던 그는 미국 시각으로 2011년 10월 12일 뉴저지주 버클리 헤이츠의 자택에서 사망한 채로 발견되었다 (향년 71세).\n업적.\n켄 톰슨(Ken Thompson) 등과 함께 최초의 유닉스(Unix) 시스템을 개발했고, 1971년 최초의 〈Unix Programmer\'s Manual〉을 썼다. 또한 C 언어를 개발한 후 브라이언 커니핸과 함께 〈C 프로그래밍 언어〉(The C Programming Language)를 기술했다. 커니핸과 〈C 프로그래밍 언어〉책을 썼기에 커니핸이 C 언어 개발에 참여한 것으로 종종 오해받으나 커니핸의 말에 따르면 자신은 C언어 개발에 참여하지 않았다고 한다.\nALTRAN, B언어, BCPL, Multics 등의 개발에도 영향을 끼친 것으로도 알려져 있다.\n1983년에 켄 톰프슨과 \"범용 운영체제 이론개발, 특히 유닉스 운영체제의 구현에 대한 공로\"로 튜링상을 수상했다.",
]

In [ ]:
messages = [
    {
        "role": "user",
        "content": f"""당신이 가진 지식을 의존하지 말고 '문서1'부터 '문서5'를 참고해서 '질문'에 대해서 답변해 주세요.:

문서5: {query_docs[4]}

문서4: {query_docs[3]}

문서3: {query_docs[2]}

문서2: {query_docs[1]}

문서1: {query_docs[0]}

질문: {query}"""
    }
]
prompt = pipe.tokenizer.apply_chat_template(messages,
                                            tokenize=False,
                                            add_generation_prompt=True)

In [ ]:
print(prompt)

### 5.4. sllm inference
- 이전 단계에서 생성한 prompt를 이용해 추론하고 결과를 확인하는 과장입니다.

In [ ]:
outputs = pipe(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)
outputs

In [ ]:
print(outputs[0]["generated_text"])

In [ ]:
print(outputs[0]["generated_text"][len(prompt):])

### 5.5. sllm chatbot
- chatbot 형식의 QA 예 입니다.

In [ ]:
# 프롬프트 생성 함수
def gen_prompt(pipe, query_docs, query):
    messages = [
        {
            "role": "user",
            "content": f"""당신이 가진 지식을 의존하지 말고 '문서1'부터 '문서5'를 참고해서 '질문'에 대해서 답변해 주세요.:

문서5: {query_docs[4]}

문서4: {query_docs[3]}

문서3: {query_docs[2]}

문서2: {query_docs[1]}

문서1: {query_docs[0]}

질문: {query}"""
        }
    ]
    prompt = pipe.tokenizer.apply_chat_template(messages,
                                                tokenize=False,
                                                add_generation_prompt=True)
    return prompt

In [ ]:
# 프롬프트 생성 및 질문을 sllm에게 묻고 결과를 리턴하는 함수
def gen_response(pipe, query_docs, query):
    prompt = gen_prompt(pipe, query_docs, query)

    outputs = pipe(
        prompt,
        do_sample=True,
        temperature=0.2,
        top_k=50,
        top_p=0.95,
        add_special_tokens=True
    )
    return outputs[0]["generated_text"][len(prompt):]

In [ ]:
# sentence transformer 검색결과
query_docs = [
    "지미 카터\n제임스 얼 “지미” 카터 주니어(, 1924년 10월 1일~)는 민주당 출신 미국의 제39대 대통령 (1977-81)이다.\n생애.\n어린 시절.\n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n조지아 공과대학교를 졸업하였고, 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 \"땅콩 농부\" (Peanut Farmer)로 알려졌다.\n정계 입문.\n1962년 조지아주 상원 의원 선거에서 낙선하였으나, 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사 선거에서 당선됐다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.\n대통령 재임.\n1976년 미합중국 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책을 내세워서 많은 지지를 받았으며 제럴드 포드 대통령을 누르고 당선되었다.\n카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.",
    "지미 카터\n1979년~1980년 대한민국의 정치적 격변기 당시의 대통령이었던 그는 이에 대해 애매한 태도를 보였고, 이는 후에 대한민국 내에서 고조되는 반미 운동의 한 원인이 됐다. 10월 26일, 박정희 대통령이 김재규 중앙정보부장에 의해 살해된 것에 대해 그는 이 사건으로 큰 충격을 받았으며, 사이러스 밴스 국무장관을 조문사절로 파견했다. 12·12 군사 반란과 5.17 쿠데타에 대해 초기에는 강하게 비난했으나, 미국 정부가 신군부를 설득하는데, 한계가 있었고 결국 묵인하는 듯한 태도를 보이게 됐다.\n퇴임 이후.\n퇴임 이후 민간 자원을 적극 활용한 비영리 기구인 카터 재단을 설립한 뒤 민주주의 실현을 위해 제 3세계의 선거 감시 활동 및 기니 벌레에 의한 드라쿤쿠르스 질병 방재를 위해 힘썼다. 미국의 빈곤층 지원 활동, 사랑의 집짓기 운동, 국제 분쟁 중재 등의 활동도 했다.\n카터는 카터 행정부 이후 미국이 북핵 위기, 코소보 전쟁, 이라크 전쟁과 같이 미국이 군사적 행동을 최후로 선택하는 전통적 사고를 버리고 군사적 행동을 선행하는 행위에 대해 깊은 유감을 표시 하며 미국의 군사적 활동에 강한 반대 입장을 보이고 있다.",
    "지미 카터\n카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.\n외교 정책.\n카터는 이집트와 이스라엘을 조정하여 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다. 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 그러나 1979년, 양국 간의 평화조약이 백악관에서 이루어졌다.\n소련과 제2차 전략 무기 제한 협상(SALT II)에 조인했다.\n카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으며, 취임 이후 계속해서 도덕정치를 내세웠다.\n임기 말, 소련의 아프가니스탄 침공 사건으로 인해 1980년 하계 올림픽에 반공국가들의 보이콧을 하였다.\n그는 주이란 미국 대사관 인질 사건의 인질 구출 실패로 인한 원인으로, 1980년 제40대 대통령 선거에서 공화당의 로널드 레이건에게 패배하며 재선에 실패하였다.\n대한민국과의 관계.\n지미 카터는 대한민국과의 관계에서도 중요한 영향을 미쳤던 대통령 중 하나다. 인권 문제와 주한미군 철수 문제로 한때 한미 관계가 불편하기도 했다. 1978년 대한민국에 대한 조선민주주의 인민공화국의 위협에 대비해 한미연합사를 창설하면서, 1982년까지 3단계에 걸쳐 주한미군을 철수하기로 했다. 그러나 주한미군사령부와 정보기관·의회의 반대에 부딪혀 주한미군은 완전철수 대신 6,000명을 감축하는 데 그쳤다. 또한 박정희 정권의 인권 문제 등과의 논란으로 불협화음을 냈으나, 1979년 6월 하순, 대한민국을 방문했는데 관계가 다소 회복되었다.",
    "노무현\n1953년에 진영대창국민학교에 입학하였고 학업 성적은 우수했으나 가난으로 결석이 잦았다고 한다. 6학년 때 담임교사의 권유로 전교 학생회장을 맡았다. 1959년 3월에 진영중학교에 진학했다. 중학교 재학 당시 노무현은 입학금이 없어 중학교는 외상으로 입학하였다고 한다. 1학년 말 제4대 정·부통령 선거를 앞두고 당시 이승만 대통령의 생일을 기념하는 교내 글짓기 대회가 열리자 노무현은 백지동맹을 일으키다가 정학을 당하였다. 집안 형편이 어려워져 중학교를 1년간 휴학한 뒤 부일장학회의 장학금을 얻어 가까스로 중학교에 다니다가 1963년에 가까스로 졸업하고 부산상고에 진학하여 1966년에 졸업하였다.\n청년기.\n고등학교 졸업 후 농업협동조합의 입사 시험에 응시했으나 낙방하고, 한 어망 제조업체에 취직하였으나 최저 생계비에도 미치지 못하는 임금과 다쳐도 치료비조차 주지 않는 고용주의 비정함에 실망하여 결국 그만두었다. 이후 막노동과 사시 공부를 병행하였다.\n1968년 군에 입대하여 제12사단 을지부대에서 육군 상병으로 만기전역하였다. 1972년 27세에 권양숙과 결혼하였고 1973년에 아들 건호를, 1975년에 딸 정연을 낳았다. 1975년 3월 30세에 제17회 사법시험에 합격하였다. 이는 4번째 도전에서 이루어진 성과였고 노무현은 제17회 사법시험에서 유일하게 고졸 출신 합격자였다. 이후 1977년 대전지방법원의 판사로 임용되었으나 5개월 만에 사직하였다.",
    "데니스 리치\n데니스 매캘리스터 리치(, 1941년 9월 9일~2011년 10월 12일)는 미국의 저명한 컴퓨터과학자이자 현대 컴퓨터과학의 선구자이다. C와 유닉스의 개발자로 알려져 있다.\n생애.\n미국의 뉴욕주 브롱크스빌(Bronxville)에서 태어났으며, 1968년 하버드 대학교에서 응용수학 박사학위를 얻었다. 1968년부터 벨 연구소 컴퓨터 연구 센터에서 일했다. 2007년 루슨트 테크놀로지의 시스템 소프트웨어 연구부장으로 은퇴했다. 홀로 살고 있던 그는 미국 시각으로 2011년 10월 12일 뉴저지주 버클리 헤이츠의 자택에서 사망한 채로 발견되었다 (향년 71세).\n업적.\n켄 톰슨(Ken Thompson) 등과 함께 최초의 유닉스(Unix) 시스템을 개발했고, 1971년 최초의 〈Unix Programmer\'s Manual〉을 썼다. 또한 C 언어를 개발한 후 브라이언 커니핸과 함께 〈C 프로그래밍 언어〉(The C Programming Language)를 기술했다. 커니핸과 〈C 프로그래밍 언어〉책을 썼기에 커니핸이 C 언어 개발에 참여한 것으로 종종 오해받으나 커니핸의 말에 따르면 자신은 C언어 개발에 참여하지 않았다고 한다.\nALTRAN, B언어, BCPL, Multics 등의 개발에도 영향을 끼친 것으로도 알려져 있다.\n1983년에 켄 톰프슨과 \"범용 운영체제 이론개발, 특히 유닉스 운영체제의 구현에 대한 공로\"로 튜링상을 수상했다.",
]

In [ ]:
while True:
    query = input('질문 > ')
    query = query.strip()
    if len(query) == 0:
        break
    result = gen_response(pipe, query_docs, query)
    print(f'답변 > {result}\n\n')

## 6. SentenceTransformer & SLLM RAG 실습 (재시작 필요)
- 다음 순서로 동작하는 RAG chatbot을 구현하세요.
  - 사용자가 질문을 입력합니다.
  - 질문에 대해서 관련된 문서를 SentenceTransformers를 이용해서 5개 구합니다.
  - 질문과 함께 문서 5개를 SLLM에 입력합니다.
  - 응답결과를 출력합니다.

### 6.1. SentenceTransformers 검색
- CHUNK_FN의 전체 문서를 사용하세요.
- SentenceTransformers를 활용한 검색 기능을 구현하세요.

In [ ]:
# chunk list


In [ ]:
# SentenceTransformer 모델 생성


In [ ]:
# 모든 chunk 사용


In [ ]:
# chunk embeddings 생성


In [ ]:
# 검색 함수 정의


In [ ]:
# 대화형 검색 (기능 확인)


### 6.2. SLLM RAG
- SLLM RAG chatbot을 구현하세요.

In [ ]:
# declare 4 bits quantize

# load 4 bits model

# load tokenizer


In [ ]:
# pipeline


In [ ]:
# 프롬프트 생성 함수


In [ ]:
# 프롬프트 생성 및 질문을 sllm에게 묻고 결과를 리턴하는 함수


In [ ]:
# 대화형 QA
while True:
    query = input('질문 > ')
    query = query.strip()
    if len(query) == 0:
        break
    # 문서 검색
    query_result = query_sentence_transformer(embed_model, chunk_embeddings, query)
    query_docs = []
    for i, score in query_result:
        chunk = chunk_list[i]
        title = chunk['metadata']['title']
        document = chunk['document']
        query_docs.append(f"{title}\n{document}")
    # sllm에게 질문
    result = gen_response(pipe, query_docs, query)
    print(f'답변 > {result}\n\n')

## 7. LangChain RAG (재시작 필요)
- LangChain을 이용하면 RAG를 훨씬 간단하게 구현할 수 있습니다.
- 이 과정을 시작하기 전 colab 세션을 다시 시작하세요.
- colab 세션을 다시 시작해야 하는 이유는 LLM의 model의 크기가 너무 크기 때문에 GPU의 메모리를 초기화 하기 위해서 입니다.

In [ ]:
# 라이브러리 설치
!pip install -qq langchain_community
!pip install -qq faiss-gpu

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.vectorstores import FAISS

In [ ]:
# Hugging Face 인증을 위해 지정
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HF_TOKEN

In [ ]:
# embedding 모델
embed_model = HuggingFaceEmbeddings(
        model_name=EMBED_MODEL_ID,
        model_kwargs={"device": "cuda"},
        encode_kwargs={"normalize_embeddings": True},
    )
embed_model

In [ ]:
# chunk list
chunk_list = []
with open(CHUNK_FN, encoding="utf-8") as f:
    chunk_list = json.load(f)
len(chunk_list)

In [ ]:
# 모든 chunk 사용
chunk_docs = []
for chunk in chunk_list:
    title = chunk['metadata']['title']
    document = chunk['document']
    chunk_docs.append(f"{title}\n{document}")
len(chunk_docs)

In [ ]:
# FAISS Vector Store에 저장
vdb = FAISS.from_texts(chunk_docs, embed_model)
vdb

In [ ]:
# Vector store-backed retriever
retriever = vdb.as_retriever(search_kwargs={"k": 5})
retriever

In [ ]:
# sllm
sllm_model = HuggingFaceEndpoint(repo_id=SLLM_MODEL_ID,
                                 max_new_tokens=1024,
                                 temperature=0.1)
sllm_model

In [ ]:
PROMPT_TEMPLATE = """당신이 가진 지식보다 아래 내용을 내용을 참고해서 '질문'에 대해서 답변해 주세요.:

내용: {context}


질문: {question}
"""

# prompt
prompt = PromptTemplate.from_template(PROMPT_TEMPLATE)
prompt

In [ ]:
# 검색 결과 형식 지정
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
# 데이터 흐름 (chain) 구성
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | sllm_model
    | StrOutputParser()
)

In [ ]:
# 기능 확인
chain.invoke("지미 카터가 졸업한 대학은 어디야?")

In [ ]:

# 대화형 QA
while True:
    query = input('질문 > ')
    query = query.strip()
    if len(query) == 0:
        break
    result = chain.invoke(query)
    print(f'답변 > {result}\n\n')